# Live ML.Net - Machine Learning 101 - Sensor MultiClassCalssification

| Versão ML.NET  |  Status                        | App Type    | Data type | Scenario            | ML Task                   | Algorithms                  |
|-------------------|-------------------------------|-------------|-----------|---------------------|---------------------------|-----------------------------|
| v1.5.4           | Up-to-date | Jupyter Notebook | .Csv files | Sensor classification | Multi-class classification | Sdca Multi-class |

Neste Notebook temos ume exemplo de uso do [ML.NET](https://github.com/dcostea/SmartFireAlarm/tree/master/SmartFireAlarm/Jupyter)  (com adpatações) baseado  na série de artigos do [Daniel Costea](https://www.linkedin.com/in/danielcostea/) no blog [jaxenter](https://jaxenter.com/author/danielcostea) para um classificador Multi-Classes.
 

## Davi Ramos -> Cientista de Dados 👋
(davi.info@gmail.com)

[![Linkedin Badge](https://img.shields.io/badge/-LinkedIn-blue?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/davi-ramos/)](https://www.linkedin.com/in/davi-ramos/)
[![Twitter Badge](https://img.shields.io/badge/-Twitter-1DA1F2?style=flat-square&logo=Twitter&logoColor=white&link=https://twitter.com/Daviinfo/)](https://twitter.com/Daviinfo/)
<a href="https://github.com/DaviRamos"><img src="https://img.shields.io/github/followers/DaviRamos.svg?label=GitHub&style=social" alt="GitHub"></a>\

In [1]:
// Instalar os Pacotes do Nuget

// ML.NET
#r "nuget:Microsoft.ML"  
   
//  DataFrame
#r "nuget:Microsoft.Data.Analysis"

//  XPlot
#r "nuget:XPlot.Plotly"   

// MathNet.Numerics
#r "nuget:MathNet.Numerics"
using MathNet.Numerics.Statistics;

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package Microsoft.ML version 1.5.4

Installed package MathNet.Numerics version 4.15.0

Installed package XPlot.Plotly version 3.0.1

In [1]:
using System;
using System.Linq;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.AspNetCore.Html;
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;
using MathNet.Numerics.Statistics;
using System.Collections.Generic;

In [1]:
MLContext mlContext = new MLContext(seed: 2020);

## Declaração dos modelos para nosso conjunto de dados

In [1]:
public class ModelInput
{
    [ColumnName("Temperature"), LoadColumn(0)]
    public float Temperature { get; set; }

    [ColumnName("Luminosity"), LoadColumn(1)]
    public float Luminosity { get; set; }

    [ColumnName("Infrared"), LoadColumn(2)]
    public float Infrared { get; set; }

    [ColumnName("Distance"), LoadColumn(3)]
    public float Distance { get; set; }

    [ColumnName("PIR"), LoadColumn(4)]
    public float PIR { get; set; }

    [ColumnName("Humidity"), LoadColumn(5)]
    public float Humidity { get; set; }

    [ColumnName("CreatedAt"), LoadColumn(6)]
    public string CreatedAt { get; set; }

    [ColumnName("Label"), LoadColumn(7)]
    public string Source { get; set; }
}

public class ModelOutput
{
    [ColumnName("PredictedLabel")]
    public string PredictedLabel;

    [ColumnName("Score")]
    public float[] Score;
}

In [1]:
// Caminho dos arquivos
var datasetPath = "./Datasets/sensors/sensors_data.csv";
var ModelPath = @"./Models/SensorModel.zip";

Error: (2,20): error CS0246: O nome do tipo ou do namespace "DataFrame" não pode ser encontrado (está faltando uma diretiva using ou uma referência de assembly?)

In [1]:
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: datasetPath,
    hasHeader: true,
    separatorChar: ',');

Carregar alguns dados estruturados do arquivo csv usando o  MLContext.

In [1]:
var shuffledData = mlContext.Data.ShuffleRows(data, seed: 2020);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.2);
var trainingData = split.TrainSet;
var testingData = split.TestSet;

É conveniente embaralhar os dados e dividi-los em duas categorias, dados de treinamento e dados de teste num subconjunto de 80% paraose dados de treinamento e o restante 20% para conjunto de dados de teste.

In [1]:
var features = mlContext.Data.CreateEnumerable<ModelInput>(trainingData, true);
var sources = features.Select(f => f.Source);
var temperatures = features.Select(f => f.Temperature);
var luminosities = features.Select(f => f.Luminosity);
var infrareds = features.Select(f => f.Infrared);
var distances = features.Select(f => f.Distance);
var pirs = features.Select(f => f.PIR);
var humidities = features.Select(f => f.Humidity);
var labels = features.Select(f => f.Source);

Os dados de treinamento do DataView não estão diretamente acessíveis, portanto, podemos criar uma coleção a partir deles e exibi-los usando o comando display. Deixe-me entrar em detalhes. Em um Notebook Jupyter, podemos usar Console.WriteLine para imprimir dados, mas adoraremos exibir o comando, pois ele pode imprimir texto, html, svn e gráficos, usando DataFrame. Vamos ter cuidado para não exibir o conjunto de dados inteiro para que possamos usar Take (10) para buscar as primeiras 10 observações.

In [1]:
display(features.Take(10)); // be carefull here not to display the entire dataset

index,Temperature,Luminosity,Infrared,Distance,PIR,Humidity,CreatedAt,Source
0,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
1,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
2,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
3,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
4,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
5,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
6,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
7,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
8,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight
9,32.5,56.93,0,33,0,13,09/09/2020 08:19:36,FlashLight


Podemos notar alguns elementos especiais na imagem acima. Uma observação é uma leitura: uma linha com um conjunto de características. Os recursos são variáveis no conjunto de dados identificadas como colunas. Um rótulo ou uma variável de destino é um tipo especial de recurso que estamos tentando prever. Qualquer recurso pode ser um rótulo, dependendo do problema que estamos resolvendo.

No próximo fórmula, os valores de x são as características e f é o nosso modelo que prevê o rótulo Y .

Y = f (x1, x2, ... xn)

É claro que não entendemos muito de olhar para os dados tabulares, mas Jupyter traz alguns tipos de diagramas excelentes com a biblioteca XPlot.Plotly, capaz de agregar os dados de uma maneira mais útil. No Histograma abaixo podemos ver as categorias:

In [1]:
var categoriesHistogram = Chart.Plot(
    new Graph.Histogram { x = labels }
  );
  display(categoriesHistogram);

In [1]:
//  Listagem 2
var segmentationDiagram = Chart.Plot(new[] {
    new Graph.Box { y = temperatures, name = "Temperature" },
    new Graph.Box { y = luminosities, name = "Luminosity" },
    new Graph.Box { y = infrareds, name = "Infrared" },
    new Graph.Box { y = distances, name = "Distance" },
    new Graph.Box { y = pirs, name = "PIR" },
    new Graph.Box { y = humidities, name = "Humidity" }
});

var layout = new Layout.Layout()
{
    title = "Box plot segmentation"
};
segmentationDiagram.WithLayout(layout);

display(segmentationDiagram);

Olhando para o diagrama acima, podemos extrair informações valiosas como:

a barra mediana da Distância é muito mais alta em comparação com os outros recursos
os valores mín-máx de temperatura e infravermelho não são uniformemente distribuídos
A temperatura tem muitos outliers
Podemos usar essas informações posteriormente para melhorar a precisão do modelo.
Para preparar os dados, temos que lembrar que lidamos com uma máquina e temos que transformar todos os dados categóricos (strings) em números usando transformadores categóricos como OneHotEncoding.

### Matriz de correlação
Pensando em dados, surge outra questão: realmente precisamos de todos os recursos? Muito provavelmente, mas alguns são menos importantes do que outros. A matriz de correlação (fig. 4) é um excelente instrumento capaz de medir a correlação entre as características da seguinte forma:

próximo a -1 ou 1 indica uma relação forte (proporcionalidade).
mais perto de 0 indica um relacionamento fraco.
0 indica nenhum relacionamento.
O código a seguir pode parecer confuso, mas precisamos preparar os dados para a matriz de correlação, nada mais é do que alinhar os valores em pares e chamar a função Correlation Pearson neles.

In [1]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared", "Distance", "PIR", "Humidity" };
var featureMatrix = new List<List<double>>();

featureMatrix.Add(temperatures.Select(Convert.ToDouble).ToList());
featureMatrix.Add(luminosities.Select(Convert.ToDouble).ToList());
featureMatrix.Add(infrareds.Select(Convert.ToDouble).ToList());
featureMatrix.Add(distances.Select(Convert.ToDouble).ToList());
featureMatrix.Add(pirs.Select(Convert.ToDouble).ToList());
featureMatrix.Add(humidities.Select(Convert.ToDouble).ToList());

// GetPearsonCorrelation

var length = featureColumns.Length;

var z = new double[length, length];
for (int x = 0; x < length; ++x)
{
  for (int y = 0; y < length - 1 - x; ++y)
  {
    var seriesA = featureMatrix[x];
    var seriesB = featureMatrix[length - 1 - y];

    var value = Correlation.Pearson(seriesA, seriesB);

    z[x, y] = value;
    z[length - 1 - y, length - 1 - x] = value;
  }

  z[x, length - 1 - x] = 1;
}

In [1]:
// Listagem 4
var correlationMatrixHeatmap = Chart.Plot(
    new Graph.Heatmap 
    {
      x = featureColumns,
      y = featureColumns.Reverse(),
      //        z = Helpers.GetPearsonCorrelation(featureMatrix),
      z = z,
      zmin = -1,
      zmax = 1
    }
  );
  
  var layout = new Layout.Layout()
  {
      autosize = "true", 
      margin =  new Graph.Margin{ l = 90 }, // fix left margin to accomodate longer labels
      title = "Matrix Correlação"
  };
  
  correlationMatrixHeatmap.WithLayout(layout);
  
  display(correlationMatrixHeatmap);

Os recursos fortemente correlacionados não transmitem informações extras, portanto, podem ser removidos (não é o caso aqui!). Por exemplo, nossos recursos mais correlacionados são Distância e Infravermelho (0,48), e Temperatura parece ser o recurso menos correlacionado em comparação com os outros recursos.

## Construir o pipeline de pré-processamento

Por convenção, o ML.NET espera encontrar a coluna Recursos (como entrada) e a coluna Rótulo (como saída), se você tiver essas colunas, não precisará fornecê-las, caso contrário, será necessário fazer algumas transformações de dados em ordem para expor essas colunas aos transformadores. Além disso, se precisarmos fazer classificações binárias ou múltiplas, temos que converter o rótulo em um número usando MapValueToKey.

Na maioria dos casos, temos mais de um recurso relevante que pode ser necessário para treinar nosso modelo e precisamos concatená-los no recurso mencionado anteriormente denominado Recursos:

In [1]:
var featureColumns = new string [] { "Temperature", "Luminosity", "Infrared", "Distance" };
var preprocessingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label") 
  .Append(mlContext.Transforms.Concatenate("Features", featureColumns));

### ML Context
Antes de prosseguir com a construção do pipeline de treinamento, vou apresentar o contêiner do catálogo MLContext. Dentro deste objeto, podemos encontrar todos os treinadores, carregadores de dados, transformadores de dados e preditores usados ​​para um grande conjunto de tarefas como: regressão, classificação, ... Muitos deles fazem parte de pacotes nuget adicionais para manter as bibliotecas principais mais leves. O parâmetro semente é útil (por exemplo, para testes de unidade) se você deseja ter um comportamento determinístico, uma vez que é usado por divisores e alguns treinadores.

### O que é um treinador de ML?
Existem vários algoritmos de treinamento para cada tipo de tarefa ML.NET disponíveis como treinadores que podem ser encontrados em seus catálogos de treinadores correspondentes. Por exemplo, Stochastic Dual Coordinated Ascent que usamos neste artigo está disponível como Sdca (para regressão), SdcaNonCalibrated e SdcaLogisticRegression (para classificação binária) e SdcaNonCalibrated e SdcaMaximumEntropy (para classificação múltipla).

In [1]:
var trainingPipeline = preprocessingPipeline.Append(mlContext.MulticlassClassification
    .Trainers.SdcaNonCalibrated ("Label", "Features"));

Não terminamos até que façamos o pós-processamento, que em nosso caso é simplesmente mapear a chave para o valor a fim de tornar a previsão legível (veja acima o mapeamento do valor para a chave no pipeline de pré-processamento).

In [1]:
var postprocessingPipeline = trainingPipeline.Append (mlContext.Transforms
    .Conversion.MapKeyToValue ("PredictedLabel"));

O pipeline de treinamento é suficiente para criar o modelo? Claro que não, precisamos alimentar alguns dados para treinar o modelo. Observe que, até chamarmos o método Fit em nosso pipeline, não obtemos nenhum dado do carregador, exceto seu esquema, e isso porque o DataView carrega lentamente (você pode pensar nele como um Linq IEnumerable).

In [1]:
var model = postprocessingPipeline.Fit(trainingData);

In [1]:
IDataView predictions = model.Transform(testingData);


## Avalie o modelo
A avaliação tem métricas semelhantes à validação, mas são muito diferentes em termos de escopo. A avaliação é feita no conjunto de dados de teste, que foi colocado de lado quando dividimos o conjunto de dados original e não participamos do processo de treinamento.

In [1]:
// Evaluate test data
var metrics = mlContext.MulticlassClassification.Evaluate(data:predictions, labelColumnName:"Label", scoreColumnName: "Score"); 

# Métricas de Avaliação
Micro-Precisão agrega as contribuições de todas as classes para calcular a métrica média. Quanto mais próximo de 1,00, melhor. Em uma tarefa de classificação de várias classes, a micro-precisão é preferível à macro-precisão se você suspeitar que pode haver desequilíbrio de classes.

Macro-precisão é a precisão média no nível da classe. A precisão para cada classe é calculada e a macro-precisão é a média dessas precisões. Quanto mais próximo de 1,00, melhor.

Log-Loss mede o desempenho de um modelo de classificação em que a entrada de previsão é um valor de probabilidade entre 0,00 e 1,00. Quanto mais próximo de 0,00, melhor. O objetivo de nossos modelos de aprendizado de máquina é minimizar esse valor.

A redução da perda de log pode ser interpretada como a vantagem do classificador sobre uma previsão aleatória. Varia de -inf a 1,00, em que 1,00 são previsões perfeitas e 0,00 indica previsões médias. Por exemplo, se o valor for igual a 0,20, pode ser interpretado como “a probabilidade de uma predição correta é 20% melhor do que a suposição aleatória”.

Matriz de confusão
Juntamente com a microprecisão, a macro-precisão e a perda de log, podemos medir a matriz de confusão. Usando o conjunto de dados de teste, podemos fazer previsões e comparar os resultados previstos com os resultados reais e podemos organizá-los em uma matriz de confusão ( fig. 6 ).

In [1]:
private static void PrintMetrics(MulticlassClassificationMetrics metrics)
{
    Console.WriteLine($"LogLoss: {metrics.LogLoss}");
    Console.WriteLine($"LogLossReduction: {metrics.LogLossReduction}");
    Console.WriteLine($"MacroAccuracy: {metrics.MacroAccuracy}");
    Console.WriteLine($"MicroAccuracy: {metrics.MicroAccuracy}");
}

In [1]:
PrintMetrics(metrics);

LogLoss: 34,53877639128314


LogLossReduction: -24,36753651662794


MacroAccuracy: 0,8119440985437092


MicroAccuracy: 0,8205128205128205


In [1]:
Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());


Confusion table
              ||================================
PREDICTED     ||     0 |     1 |     2 |     3 | Recall
TRUTH         ||================================
0.   Infrared ||    32 |     0 |     1 |     4 | 0,8649
1.   DayLight ||     1 |    34 |     2 |     9 | 0,7391
2.    Lighter ||     9 |     0 |    33 |     3 | 0,7333
3. FlashLight ||     4 |     2 |     0 |    61 | 0,9104
              ||================================
Precision     ||0,6957 |0,9444 |0,9167 |0,7922 |



No diagrama anterior, podemos ver que a classe FlashLight foi prevista corretamente (como fonte FlashLight) 63 vezes, incorretamente como Dia uma vez e Lighter uma vez também, resultando em uma taxa de precisão de 0,9189.

In [1]:
List<string> categories = new List<string> {"Temperature", "Luminosity", "Infrared", "Distance"};
Formatter.Register<ConfusionMatrix>((df, writer) =>
{
    var cssFirstColor = "background-color: gray; ";
    var cssSecondColor = "background-color: blue; ";
    var cssTransparent = "background-color: transparent";
    var cssBold = "font-weight: bold; ";
    var cssPadding = "padding: 8px; ";
    var cssCenterAlign = "text-align: center; ";
    var cssTable = "margin: 50px; ";
    var cssTitle  = cssPadding +  cssFirstColor;
    var cssHeader = cssPadding + cssBold + cssSecondColor;
    var cssCount = cssPadding;
    var cssFormula = cssPadding + cssSecondColor;
    
    var rows = new List<IHtmlContent>();
    
    // header
    var cells = new List<IHtmlContent>();
    cells.Add(td[rowspan: 2, colspan: 2, style: cssTitle + cssCenterAlign]("Confusion Matrix"));
    cells.Add(td[colspan: df.Counts.Count, style: cssTitle + cssCenterAlign]("Predicted"));
    cells.Add(td[style: cssTitle](""));
    rows.Add(tr[style: cssTransparent](cells));

    // features header
    cells = new List<IHtmlContent>();
    for (int j = 0; j < df.Counts.Count; j++)
    {
        cells.Add(td[style: cssHeader](categories.ToList()[j]));
    }
    rows.Add(tr[style: cssTransparent](cells));
    cells.Add(td[style: cssTitle]("Recall"));

    // values
    for (int i = 0; i < df.NumberOfClasses; i++)
    {
        cells = new List<IHtmlContent>();
        if (i == 0)
        {
            cells.Add(td[rowspan: df.Counts.Count, style: cssTitle]("Truth"));
        }
        cells.Add(td[style: cssHeader](categories.ToList()[i]));
        for (int j = 0; j < df.NumberOfClasses; j++)
        {
            cells.Add(td[style: cssCount](df.Counts[i][j]));
        }
        cells.Add(td[style: cssFormula](Math.Round(df.PerClassRecall[i], 4)));
        rows.Add(tr[style: cssTransparent](cells));
    }

    //footer
    cells = new List<IHtmlContent>();
    cells.Add(td[colspan: 2, style: cssTitle]("Precision"));
    for (int j = 0; j < df.Counts.Count; j++)
    {
        cells.Add(td[style: cssFormula](Math.Round(df.PerClassPrecision[j], 4)));
    }
    cells.Add(td[style: cssFormula]("total = " + df.Counts.Sum(x => x.Sum())));
    rows.Add(tr[style: cssTransparent](cells));

    writer.Write(table[style: cssTable](tbody(rows)));
}, "text/html");

display(metrics.ConfusionMatrix);

Confusion Matrix Predicted Temperature Luminosity Infrared Distance Recall Truth Temperature 32 0 1 4 0.8649 Luminosity 1 34 2 9 0.7391 Infrared 9 0 33 3 0.7333 Distance 4 2 0 61 0.9104 Precision 0.6957 0.9444 0.9167 0.7922 total = 195

## Salvar o modelo
Depois de experimentar diferentes conjuntos de recursos, instrutores e parâmetros para construir um modelo de aprendizado de máquina, escolhemos aquele que melhor se adapta às nossas necessidades. Muito provavelmente, precisamos do modelo em um cenário de produção, então temos que persistir o modelo em um arquivo físico em formato nativo (ML.NET) ou em formato ONNX (que é um formato portátil desenvolvido pela Microsoft e Facebook e adotado por outros grandes players também). Sem surpresa, se você der uma olhada no arquivo (para o formato nativo), você perceberá que é um zip de arquivos de texto contendo números.

Salvando o modelo em formato nativo:

In [1]:
mlContext.Model.Save(model, trainingData.Schema, ModelPath);

In [1]:
// Carregar o modelo
DataViewSchema modelSchema;
var trainedModel = mlContext.Model.Load(ModelPath,  out modelSchema);

In [1]:
// Realizar predição 
var sampleData = new ModelInput
{
    Temperature = 24.77f,
    Luminosity = 64.06f,
    Infrared = 0,
    Distance = 55,   
    PIR = 0,
    Humidity = 11,
    CreatedAt = DateTime.Now.ToString("dd/MM/yyyy hh:mm:ss")
};

var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);

var predicted = predictor.Predict(sampleData);

Console.WriteLine(predicted.PredictedLabel);

FlashLight
